<a href="https://colab.research.google.com/github/HansDampf37/Kaggle/blob/main/Kaggle_OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [3]:
df = pd.read_csv('data/ocr/train.csv')
y = df['label']
X = df.drop('label', axis=1)
X = X.values.reshape(-1, 28, 28)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(33600, 28, 28)
(8400, 28, 28)
(33600,)
(8400,)


In [4]:
class OCR(nn.Module):
    def __init__(self):
        super(OCR, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) # 28
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)) # 14
        self.conv2 = nn.Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)) # 7
        self.conv3 = nn.Conv2d(16, 16, kernel_size=(2, 2), stride=(1, 1)) # 6
        self.pool3 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2)) # 3
        self.fc1 = nn.Linear(3 * 3 * 16, 10)
        self.fc2 = nn.Linear(10, 10)
        self.relu = nn.ReLU()
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.parameters(), lr=0.001)


    def forward(self, x):
        x = self.pool1(self.relu(self.conv1(x)))
        x = self.pool2(self.relu(self.conv2(x)))
        x = self.pool3(self.relu(self.conv3(x)))
        x = nn.Flatten()(x)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def score(self, X, y):
        self.eval()
        with torch.no_grad():
            output = self(torch.from_numpy(X).unsqueeze(1).float())
            _, predicted = torch.max(output.data, 1)
            total = y.shape[0]
            correct = (predicted == torch.from_numpy(y.values)).sum().item()
            return correct / total

ocr = OCR()

In [5]:
def train(model, train_loader, epochs):
    train_accs = []
    test_accs = []
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            outputs = model(inputs)
            loss = model.criterion(outputs, labels)
            
            # Backward pass and optimization
            model.optimizer.zero_grad()
            loss.backward()
            model.optimizer.step()

        # Print statistics
        test_acc = model.score(X_test, y_test)
        train_acc = model.score(X_train, y_train)
        test_accs.append(test_acc)
        train_accs.append(train_acc)
        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(train_loader):.4f}, Test-Acc: {test_acc:.4f}, Train-Acc: {train_acc:.4f}')


dataset = TensorDataset(torch.from_numpy(X_train).unsqueeze(1).float(), torch.from_numpy(y_train.values).long())
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)
train(ocr, train_loader, epochs=10)


Epoch [1/10], Loss: 0.0000, Test-Acc: 0.9268, Train-Acc: 0.9325
Epoch [2/10], Loss: 0.0000, Test-Acc: 0.9529, Train-Acc: 0.9606
Epoch [3/10], Loss: 0.0000, Test-Acc: 0.9554, Train-Acc: 0.9649
Epoch [4/10], Loss: 0.0000, Test-Acc: 0.9626, Train-Acc: 0.9693
Epoch [5/10], Loss: 0.0000, Test-Acc: 0.9695, Train-Acc: 0.9759
Epoch [6/10], Loss: 0.0000, Test-Acc: 0.9702, Train-Acc: 0.9786
Epoch [7/10], Loss: 0.0000, Test-Acc: 0.9731, Train-Acc: 0.9834
Epoch [8/10], Loss: 0.0000, Test-Acc: 0.9742, Train-Acc: 0.9827
Epoch [9/10], Loss: 0.0000, Test-Acc: 0.9724, Train-Acc: 0.9836
Epoch [10/10], Loss: 0.0000, Test-Acc: 0.9740, Train-Acc: 0.9859


In [6]:
ocr.score(X_test, y_test)

0.974047619047619

In [7]:
df = pd.read_csv('data/ocr/test.csv')
X = df.values.reshape(-1, 28, 28)
output = ocr(torch.from_numpy(X).unsqueeze(1).float())

submission = pd.DataFrame()
submission['ImageId'] = df.index + 1
submission['label'] = torch.argmax(output, dim=1).numpy()
submission.to_csv('data/ocr/submission.csv', index=False)

In [9]:
submission

,ImageId,label
0,1,2
1,2,0
2,3,9
3,4,5
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9
